In [1]:
from os import chdir 
chdir('./lib')

In [2]:
!pwd

/Users/ginodefalco/dsi/dsi_repo/DSI_SM_3/projects/project-05/lib


In [3]:
from project_5 import general_process, load_data_from_database, make_data_dict, general_model, general_transformer

# Step 3 - Build Model

### Domain and Data

MADELON is an artificial dataset, which was part of the NIPS 2003 feature selection challenge. This is a two-class classification problem with continuous input variables. The difficulty is that the problem is multivariate and highly non-linear.
MADELON is an artificial dataset containing data points grouped in 32 clusters placed on the vertices of a five dimensional hypercube and randomly labeled +1 or -1. The five dimensions constitute 5 informative features. 15 linear combinations of those features were added to form a set of 20 (redundant) informative features. Based on those 20 features one must separate the examples into the 2 classes (corresponding to the +-1 labels). A number of distractor feature called 'probes' were added having no predictive power. The order of the features and patterns were randomized.

### Problem Statement

We are going to further filter our features using a simple Grid Search Cross Validation model.

### Solution Statement

In order to do this, we are going to run a Grid Search Cross Validation through our data
We will construct a Pipeline that uses SelectKBest to transform data
We will construct a Pipeline that uses LogisticRegression to model data
We will construct a Pipeline that uses KNearestNeighbors to model data
And Gridsearch optimal parameters for logistic regression and KNN

### Metric

Our metric is our accuracy score on our feature selection

### Benchmark

Our benchmark hasn't changed since step 1.

## Implementation

Implement the following code pipeline using the functions you write in `lib/project_5.py`.

<img src="assets/build_model.png" width="600px">

In [4]:
# Let's use the function we wrote to load our data into the dataframe
df = load_data_from_database()
df.shape

(2000, 502)

In [5]:
# Let's get info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 502 entries, index to label
dtypes: int64(502)
memory usage: 7.7 MB


In [6]:
# Describe the Data
df.describe()

,index,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_491,feat_492,feat_493,feat_494,feat_495,feat_496,feat_497,feat_498,feat_499,label
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000
mean,999.500000,481.722500,483.452500,510.166000,483.384500,501.612500,479.259000,480.109500,476.565000,486.793500,...,478.811500,486.356500,496.565500,493.49950,510.893000,478.219500,483.309000,507.977000,490.266000,0.00000
std,577.494589,6.421769,30.186294,38.899165,9.059895,41.389418,6.795956,40.575925,1.384461,15.043836,...,4.011735,23.967366,127.635442,34.81902,37.459353,5.880613,13.559847,37.224297,25.825273,1.00025
min,0.000000,462.000000,381.000000,370.000000,453.000000,371.000000,459.000000,334.000000,471.000000,430.000000,...,463.000000,391.000000,130.000000,368.00000,398.000000,457.000000,435.000000,363.000000,403.000000,-1.00000
25%,499.750000,477.000000,464.000000,485.000000,477.000000,475.000000,475.000000,452.750000,476.000000,477.000000,...,476.000000,471.000000,404.000000,470.00000,486.000000,474.000000,474.000000,482.000000,473.000000,-1.00000
50%,999.500000,482.000000,483.000000,510.500000,483.000000,500.000000,479.000000,480.000000,477.000000,487.000000,...,479.000000,486.000000,504.000000,492.00000,511.000000,478.000000,483.000000,508.000000,490.000000,0.00000
75%,1499.250000,486.000000,503.000000,536.000000,490.000000,528.000000,484.000000,506.250000,477.000000,496.250000,...,481.000000,502.000000,586.000000,517.00000,535.000000,482.000000,492.000000,533.000000,507.250000,1.00000
max,1999.000000,503.000000,600.000000,654.000000,519.000000,688.000000,505.000000,611.000000,481.000000,536.000000,...,497.000000,566.000000,920.000000,615.00000,661.000000,500.000000,535.000000,644.000000,583.000000,1.00000


In [7]:
# Let's load our make_data_dict function to 
df = make_data_dict(df, random_state=0)
df

{'X_test':       index  feat_000  feat_001  feat_002  feat_003  feat_004  feat_005  \
 1618   1618       464       463       526       487       533       481   
 1905   1905       493       523       558       494       526       476   
 1751   1751       479       469       504       485       482       481   
 283     283       482       464       553       478       494       470   
 1313   1313       474       502       508       488       593       484   
 1803   1803       486       453       582       499       472       487   
 1262   1262       477       510       467       488       490       493   
 613     613       481       469       518       485       547       473   
 1000   1000       482       538       473       485       576       479   
 724     724       483       507       508       496       484       479   
 1601   1601       489       457       489       475       452       475   
 1567   1567       478       460       545       474       544       478   
 7

In [36]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline


In [46]:
# We will scale our data with Standard Scaler
df = general_transformer(StandardScaler(), df)
df

{'X_test': array([[ 1.05367159, -2.77190284, -0.6814072 , ..., -1.80218292,
          0.23585583,  1.07256347],
        [ 1.54990462,  1.77014611,  1.32233873, ...,  0.50468266,
         -0.13487564, -0.82584753],
        [ 1.28363323, -0.42256717, -0.4810326 , ...,  0.95117277,
          0.92435712, -1.01956294],
        ..., 
        [-0.74279572,  0.67378947,  0.92158954, ...,  1.02558779,
         -0.21431809,  0.49141725],
        [ 0.40874155, -1.20567906,  0.52084036, ...,  0.65351269,
         -2.1738987 , -0.98081986],
        [ 1.17470403,  0.83041185, -1.3493225 , ...,  0.35585262,
          0.36825992,  0.18147259]]),
 'X_train': array([[ 1.52569813, -0.57918955,  1.0551726 , ..., -0.16505251,
         -0.47912628, -1.523223  ],
        [ 1.00698764,  0.20392234, -1.14894791, ..., -0.61154262,
         -1.16762758, -1.44573684],
        [-1.03846069,  0.51716709, -1.3493225 , ...,  0.57909768,
         -0.32024137, -0.51590288],
        ..., 
        [ 1.36489788, -2.145413

In [47]:
# construct a Pipeline that uses SelectKBest to transform data
from sklearn.feature_selection import SelectKBest
df = general_transformer(SelectKBest(f_classif, k='all'), df)

In [48]:
df

{'X_test': array([[ 1.05367159, -2.77190284, -0.6814072 , ..., -1.80218292,
          0.23585583,  1.07256347],
        [ 1.54990462,  1.77014611,  1.32233873, ...,  0.50468266,
         -0.13487564, -0.82584753],
        [ 1.28363323, -0.42256717, -0.4810326 , ...,  0.95117277,
          0.92435712, -1.01956294],
        ..., 
        [-0.74279572,  0.67378947,  0.92158954, ...,  1.02558779,
         -0.21431809,  0.49141725],
        [ 0.40874155, -1.20567906,  0.52084036, ...,  0.65351269,
         -2.1738987 , -0.98081986],
        [ 1.17470403,  0.83041185, -1.3493225 , ...,  0.35585262,
          0.36825992,  0.18147259]]),
 'X_train': array([[ 1.52569813, -0.57918955,  1.0551726 , ..., -0.16505251,
         -0.47912628, -1.523223  ],
        [ 1.00698764,  0.20392234, -1.14894791, ..., -0.61154262,
         -1.16762758, -1.44573684],
        [-1.03846069,  0.51716709, -1.3493225 , ...,  0.57909768,
         -0.32024137, -0.51590288],
        ..., 
        [ 1.36489788, -2.145413

In [80]:
# Running a general logistic regression with LASSO as the penalty
df = general_model(LogisticRegression(C=1, penalty='l1'), df)
df

{'X_test': array([[ 1.05367159, -2.77190284, -0.6814072 , ..., -1.80218292,
          0.23585583,  1.07256347],
        [ 1.54990462,  1.77014611,  1.32233873, ...,  0.50468266,
         -0.13487564, -0.82584753],
        [ 1.28363323, -0.42256717, -0.4810326 , ...,  0.95117277,
          0.92435712, -1.01956294],
        ..., 
        [-0.74279572,  0.67378947,  0.92158954, ...,  1.02558779,
         -0.21431809,  0.49141725],
        [ 0.40874155, -1.20567906,  0.52084036, ...,  0.65351269,
         -2.1738987 , -0.98081986],
        [ 1.17470403,  0.83041185, -1.3493225 , ...,  0.35585262,
          0.36825992,  0.18147259]]),
 'X_train': array([[ 1.52569813, -0.57918955,  1.0551726 , ..., -0.16505251,
         -0.47912628, -1.523223  ],
        [ 1.00698764,  0.20392234, -1.14894791, ..., -0.61154262,
         -1.16762758, -1.44573684],
        [-1.03846069,  0.51716709, -1.3493225 , ...,  0.57909768,
         -0.32024137, -0.51590288],
        ..., 
        [ 1.36489788, -2.145413

In [49]:
# Constructing a pipeline
this_pipeline = (SelectKBest(f_classif, k=5),
                 KNeighborsClassifier(n_neighbors=17))
anova_classifier = make_pipeline(*this_pipeline)

In [87]:
# Running the anova classifier model
df = general_model(anova_classifier, df)
df

{'X_test': array([[ 1.05367159, -2.77190284, -0.6814072 , ..., -1.80218292,
          0.23585583,  1.07256347],
        [ 1.54990462,  1.77014611,  1.32233873, ...,  0.50468266,
         -0.13487564, -0.82584753],
        [ 1.28363323, -0.42256717, -0.4810326 , ...,  0.95117277,
          0.92435712, -1.01956294],
        ..., 
        [-0.74279572,  0.67378947,  0.92158954, ...,  1.02558779,
         -0.21431809,  0.49141725],
        [ 0.40874155, -1.20567906,  0.52084036, ...,  0.65351269,
         -2.1738987 , -0.98081986],
        [ 1.17470403,  0.83041185, -1.3493225 , ...,  0.35585262,
          0.36825992,  0.18147259]]),
 'X_train': array([[ 1.52569813, -0.57918955,  1.0551726 , ..., -0.16505251,
         -0.47912628, -1.523223  ],
        [ 1.00698764,  0.20392234, -1.14894791, ..., -0.61154262,
         -1.16762758, -1.44573684],
        [-1.03846069,  0.51716709, -1.3493225 , ...,  0.57909768,
         -0.32024137, -0.51590288],
        ..., 
        [ 1.36489788, -2.145413

In [51]:
# Setting up pipeline using KBest and KNeighbors
this_pipeline = (SelectKBest(f_classif, k=5),
                 KNeighborsClassifier(n_neighbors=17)
)

In [52]:
# Using KBest and KNN as estimators
estimators = [('selectkbest', SelectKBest(f_classif, k=5)),
              ('classifier', KNeighborsClassifier(n_neighbors=17))]

In [53]:
pipe = Pipeline(estimators)
via_make = make_pipeline(*this_pipeline)
print pipe
print via_make

Pipeline(steps=[('selectkbest', SelectKBest(k=5, score_func=<function f_classif at 0x1149729b0>)), ('classifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=17, p=2,
           weights='uniform'))])
Pipeline(steps=[('selectkbest', SelectKBest(k=5, score_func=<function f_classif at 0x1149729b0>)), ('kneighborsclassifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=17, p=2,
           weights='uniform'))])


In [60]:
print('As a List')

print('\nIndexed as a List')
print(pipe.steps[0])

print('\nAs a Dictionary')
print(pipe.named_steps['selectkbest'])

As a List

Indexed as a List
('selectkbest', SelectKBest(k=5, score_func=<function f_classif at 0x1149729b0>))

As a Dictionary
SelectKBest(k=5, score_func=<function f_classif at 0x1149729b0>)


In [55]:
pipe.set_params(classifier__n_jobs=-1)
pipe

Pipeline(steps=[('selectkbest', SelectKBest(k=5, score_func=<function f_classif at 0x1149729b0>)), ('classifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=17, p=2,
           weights='uniform'))])

In [56]:
# Setting params
params = dict(selectkbest__k=range(5,15),
              classifier__p=[1,2])

params

{'classifier__p': [1, 2],
 'selectkbest__k': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]}

In [86]:
# Setting up Grid Search
pipe_grid_search = GridSearchCV(pipe, param_grid=params, cv=5)
pipe_grid_search

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('selectkbest', SelectKBest(k=5, score_func=<function f_classif at 0x1149729b0>)), ('classifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=17, p=2,
           weights='uniform'))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'selectkbest__k': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'classifier__p': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [91]:
# Grid Search
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
clf = general_model(GridSearchCV(LogisticRegression(penalty='l2'), param_grid))
clf
GridSearchCV(cv=None,
       estimator=LogisticRegression(C=1.0, intercept_scaling=1, dual=False, fit_intercept=True,
       penalty='l2', tol=0.0001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, verbose=0)


GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

# Step 3 Conclusion:

We constructed a Pipeline that uses SelectKBest to transform data
We constructed a Pipeline that uses LogisticRegression to model data
We constructed a Pipeline that uses KNearestNeighbors to model data
Finally, we used Gridsearch optimal parameters for logistic regression and KNN

We scored 70% Train and 64% Test which was another improvement